### Load Arduino readings

In [119]:
import glob
import math

import pandas as pd
l = [pd.read_csv(filename, names=["Humidity", "Temperature", "TemperatureF", "Lux", "Test","Ts", "DateTime"]) for filename in glob.glob("./data/readings/*.TXT")]
dfreadings = pd.concat(l, axis=0)

In [120]:
l

[   Humidity  Temperature  TemperatureF    Lux  Test  Ts             DateTime
 0      36.9         21.7         71.06  484.0   415   2  2022-12-12T13:02:04
 1      36.8         21.7         71.06  489.0   641  36  2022-12-12T13:03:04
 2      37.6         21.8         71.24  496.0   426   3  2022-12-12T13:04:04,
    Humidity  Temperature  TemperatureF    Lux  Test  Ts             DateTime
 0      37.0         21.8         71.24  495.0   400   0  2022-12-12T13:05:31
 1      36.6         21.7         71.06  493.0   432   4  2022-12-12T13:06:31,
     Humidity  Temperature  TemperatureF     Lux  Test   Ts  \
 0       35.1         21.7         71.06   841.0   400    0   
 1       35.0         21.5         70.70   868.0   400    0   
 2       35.0         21.5         70.70  1101.0   400    0   
 3       35.2         21.5         70.70  1085.0   400    0   
 4       35.1         21.5         70.70  1048.0   405    0   
 5       35.1         21.4         70.52  1075.0   411    1   
 6       35

In [121]:
dfreadings.head(n=100)

,Humidity,Temperature,TemperatureF,Lux,Test,Ts,DateTime
0,36.9,21.7,71.06,484.0,415,2,2022-12-12T13:02:04
1,36.8,21.7,71.06,489.0,641,36,2022-12-12T13:03:04
2,37.6,21.8,71.24,496.0,426,3,2022-12-12T13:04:04
0,37.0,21.8,71.24,495.0,400,0,2022-12-12T13:05:31
1,36.6,21.7,71.06,493.0,432,4,2022-12-12T13:06:31
...,...,...,...,...,...,...,...
1,35.7,19.7,67.46,495.0,400,0,2022-12-13T09:57:09
2,35.5,19.7,67.46,518.0,400,0,2022-12-13T09:58:09
3,35.5,19.8,67.64,520.0,400,0,2022-12-13T09:59:09
4,35.4,19.9,67.82,522.0,400,0,2022-12-13T10:00:09


### Load HRV data

In [122]:
import pandas as pd
import os

txtfolder = r'./Data/HRV/Niek_Snijders/' #Change to your folder path

#Find the textfiles
textfiles = []
for root, folder, files in os.walk(txtfolder):
    for file in files:
        if file.endswith('.txt'):
            fullname = os.path.join(root, file)
            textfiles.append(fullname)
textfiles.sort() #Sort the filesnames

#Read each of them to a dataframe
for filenum, file in enumerate(textfiles, 1):
    if filenum==1:
        df = pd.read_csv(file, names=['RR'], delim_whitespace=True)
        date = os.path.basename(file).split(".")[0].split(" ")[0]
        time = os.path.basename(file).split(".")[0].split(" ")[1]
        split_date = date.split("-")
        split_time = time.split("-")
        df['DateTime']=split_date[2]+"-"+split_date[1]+"-"+split_date[0]+ " " + split_time[0] + ":" + split_time[1] + ":" + split_time[2]
        df['DateTime'] = pd.to_datetime(df['DateTime'])
    else:
        tempdf = pd.read_csv(file, names=['RR'], delim_whitespace=True)
        date = os.path.basename(file).split(".")[0].split(" ")[0]
        time = os.path.basename(file).split(".")[0].split(" ")[1]
        split_date = date.split("-")
        split_time = time.split("-")
        tempdf['DateTime']=split_date[2]+"-"+split_date[1]+"-"+split_date[0]+ " " + split_time[0] + ":" + split_time[1] + ":" + split_time[2]
        tempdf['DateTime'] = pd.to_datetime(tempdf['DateTime'])
        df = pd.concat([df, tempdf], ignore_index=True)

In [123]:
df.head()

,RR,DateTime
0,776,2022-11-17 15:17:06
1,758,2022-11-17 15:17:06
2,737,2022-11-17 15:17:06
3,711,2022-11-17 15:17:06
4,738,2022-11-17 15:17:06


In [124]:
def calculateHRV(df):
    array = df[["RR"]].to_numpy()
    sumOfSuccessiveDifference = 0.0
    for idx, x in enumerate(array):
        if x != array[-1]:
            sumOfSuccessiveDifference += (array[idx] - array[idx + 1])**2
    return math.sqrt(sumOfSuccessiveDifference/len(array))

In [125]:
dfHRVNiek = df.groupby(["DateTime"]).apply(lambda x: calculateHRV(x)).astype(float)

In [126]:
dfHRVNiek.head(n=20)

DateTime
2022-01-12 14:35:12    92.179360
2022-06-12 15:52:08    63.953953
2022-06-12 15:54:01    78.683995
2022-07-12 10:19:15    45.998991
2022-07-12 12:44:46    59.847481
2022-07-12 13:57:29    48.507507
2022-11-17 15:17:06    42.056698
2022-11-18 11:00:00    42.440603
2022-11-24 12:01:05    45.918517
2022-11-30 11:07:17    36.091961
2022-11-30 11:49:08    52.136928
2022-12-13 10:26:14    16.066151
dtype: float64

### Load selfreporting data

In [127]:
import pandas as pd
l = [pd.read_csv(filename) for filename in glob.glob("./data/selfreporting/*.csv")]
dfself = pd.concat(l, axis=0)

In [128]:
dfself.head(n=40)

,Air Qualityall Good,Beverage,Cloth 1,Cloth 2,Cloth 3,Cloth 4,Cloth 5,Cloth 6,Cloth 7,Duration Of Location,...,Location,Mode Of Transport,Mood,Smelly,Stuffy,Suffocating,Thermal Comfort,Thermal Preference,Timestamp,User Name
0,YES,2,NO,YES,YES,NO,NO,NO,NO,1-2 hours,...,LA Explora floor 2,1,3,NO,NO,NO,5,1,2022/12/7 2022-12-07 10:42:04.299,job@email.com
1,YES,1,NO,YES,YES,NO,NO,NO,NO,more than 3 hours,...,LA402,1,3,NO,NO,NO,4,2,2022/12/7 2022-12-07 13:51:00.415,job@email.com
2,YES,2,NO,YES,YES,NO,NO,NO,NO,more than 3 hours,...,LA402,1,4,NO,NO,NO,5,2,2022/12/7 2022-12-07 14:04:39.307,job@email.com
3,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA222,1,4,NO,NO,NO,5,1,2022/12/12 2022-12-12 13:34:15.607,job@email.com
4,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,NO,NO,5,1,2022/12/12 2022-12-12 13:42:56.603,job@email.com
5,NO,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,YES,NO,5,1,2022/12/12 2022-12-12 13:56:09.337,job@email.com
6,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,NO,NO,4,1,2022/12/12 2022-12-12 14:02:44.077,job@email.com
7,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,NO,NO,4,2,2022/12/12 2022-12-12 14:11:54.980,job@email.com
8,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,3,NO,NO,NO,4,2,2022/12/12 2022-12-12 14:22:24.339,job@email.com
9,NO,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,3,NO,YES,NO,4,1,2022/12/12 2022-12-12 14:34:48.558,job@email.com


# Data preprocessing

In [129]:
df = dfself
df

,Air Qualityall Good,Beverage,Cloth 1,Cloth 2,Cloth 3,Cloth 4,Cloth 5,Cloth 6,Cloth 7,Duration Of Location,...,Location,Mode Of Transport,Mood,Smelly,Stuffy,Suffocating,Thermal Comfort,Thermal Preference,Timestamp,User Name
0,YES,2,NO,YES,YES,NO,NO,NO,NO,1-2 hours,...,LA Explora floor 2,1,3,NO,NO,NO,5,1,2022/12/7 2022-12-07 10:42:04.299,job@email.com
1,YES,1,NO,YES,YES,NO,NO,NO,NO,more than 3 hours,...,LA402,1,3,NO,NO,NO,4,2,2022/12/7 2022-12-07 13:51:00.415,job@email.com
2,YES,2,NO,YES,YES,NO,NO,NO,NO,more than 3 hours,...,LA402,1,4,NO,NO,NO,5,2,2022/12/7 2022-12-07 14:04:39.307,job@email.com
3,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA222,1,4,NO,NO,NO,5,1,2022/12/12 2022-12-12 13:34:15.607,job@email.com
4,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,NO,NO,5,1,2022/12/12 2022-12-12 13:42:56.603,job@email.com
5,NO,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,YES,NO,5,1,2022/12/12 2022-12-12 13:56:09.337,job@email.com
6,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,NO,NO,4,1,2022/12/12 2022-12-12 14:02:44.077,job@email.com
7,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,4,NO,NO,NO,4,2,2022/12/12 2022-12-12 14:11:54.980,job@email.com
8,YES,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,3,NO,NO,NO,4,2,2022/12/12 2022-12-12 14:22:24.339,job@email.com
9,NO,2,NO,NO,YES,NO,NO,NO,YES,more than 3 hours,...,LA223,1,3,NO,YES,NO,4,1,2022/12/12 2022-12-12 14:34:48.558,job@email.com


In [130]:
def bool_to_num(val):
    if val.lower() == "yes": return 1
    return 0

In [131]:
columns = ["Air Qualityall Good", "Smelly", "Stuffy", "Suffocating", "Humid"]
for i in columns:
    df[i] = df[i].apply(lambda x: bool_to_num(x))

for i in range(1, 8):
    column = f"Cloth {i}"
    df[column] = df[column].apply(lambda x: bool_to_num(x))

dummies = ["Duration Of Location", "Eat Recent", "Location"]
df = pd.get_dummies(df, columns=dummies)
df

,Air Qualityall Good,Beverage,Cloth 1,Cloth 2,Cloth 3,Cloth 4,Cloth 5,Cloth 6,Cloth 7,Humid,...,Duration Of Location_less than an hour,Duration Of Location_more than 3 hours,Eat Recent_30 mins ago,Eat Recent_One hour ago,Eat Recent_Two hours ago,Location_LA Explora floor 2,Location_LA222,Location_LA223,Location_LA402,Location_LD025
0,1,2,0,1,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1,1,0,1,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
2,1,2,0,1,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
3,1,2,0,0,1,0,0,0,1,0,...,0,1,1,0,0,0,1,0,0,0
4,1,2,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,0
5,0,2,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,0
6,1,2,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,0
7,1,2,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,0
8,1,2,0,0,1,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
9,0,2,0,0,1,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
